In [1]:

import numpy as np
import pandas as pd
from typing import List, Dict
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel



In [2]:
songs = pd.read_csv('Song_data.csv')
songs.head()



,artist,song,link,text,user_id,frequency,song_id
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \nAnd...",T6ylNteA,1.0,7M1OJTM2
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \nTouch me gentl...",0h29M6kv,1.0,DT69RI32
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \nWhy I had t...,Kk3gE2bG,1.0,5KELUYVE
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...,rxtykVT6,4.0,IMMIWPK1
4,Blur,Bang,/b/blur/bang_20021016.html,Sitting in an S.D.T. \nWaiting for an undergr...,hwErMKpU,4.0,IMMIWPK1


In [3]:
class SongRecommender:
    def __init__(self, song_data: pd.DataFrame):
        """
        Initializes the recommender system with song data.

        :param song_data: DataFrame containing song information.
        """
        self.song_data = song_data
        self.vectorizer = TfidfVectorizer(stop_words='english')
        self.tfidf_matrix = self._fit_tfidf()

    def _fit_tfidf(self) -> np.ndarray:
        """
        Fits the TF-IDF model on the song descriptions.

        :return: The TF-IDF matrix.
        """
        return self.vectorizer.fit_transform(self.song_data['text'])

    def get_similar_songs(self, song_title: str, top_n: int = 5) -> List[Dict[str, str]]:
        """
        Recommends a list of similar songs based on the provided song title.

        :param song_title: Title of the song for which recommendations are needed.
        :param top_n: Number of recommendations to return.
        :return: List of recommended songs with their similarity scores.
        """
        try:
            idx = self.song_data.index[self.song_data['song'] == song_title][0]
            cosine_similarities = linear_kernel(self.tfidf_matrix[idx:idx+1], self.tfidf_matrix).flatten()
            related_songs_indices = cosine_similarities.argsort()[-top_n-1:-1][::-1]
            return [
                {"song": self.song_data['song'].iloc[i], "similarity_score": cosine_similarities[i]}
                for i in related_songs_indices
            ]
        except IndexError:
            print("Song not found in the dataset.")
            return []


In [4]:
recommender = SongRecommender(songs)
recommendation = {
    "song": songs['song'].iloc[0],  # Selects the first song in the dataset
    "number_songs": 4  # Number of recommendations to retrieve
}
recommendations = recommender.get_similar_songs(recommendation['song'], recommendation['number_songs'])



In [ ]:
print(f"The {recommendation['number_songs']} recommended songs for {recommendation['song']} are:")
for i, rec in enumerate(recommendations, 1):
    print(f"Number {i}:\n{rec['song']} with {rec['similarity_score']:.3f} similarity score\n--------------------")

The 4 recommended songs for Ahe's My Kind Of Girl are:
Number 1:
What Kind Of Girl with 0.453 similarity score
--------------------
Number 2:
The Kind Of Girl I Could Love with 0.412 similarity score
--------------------
Number 3:
Girl (Why You Wanna Make Me Blue) with 0.357 similarity score
--------------------
Number 4:
Girl with 0.334 similarity score
--------------------


In [ ]:
recommendation2 = {
    "song": songs['song'].iloc[990],  # Selects another song in the dataset (for example, the third one)
    "number_songs": 10  # Number of recommendations to retrieve
}
recommendations2 = recommender.get_similar_songs(recommendation2['song'], recommendation2['number_songs'])

# Print out the recommendations for the second song
print(f"The {recommendation2['number_songs']} recommended songs for {recommendation2['song']} are:")
for i, rec in enumerate(recommendations2, 1):
    print(f"Number {i}:\n{rec['song']} with {rec['similarity_score']:.3f} similarity score\n--------------------")

The 10 recommended songs for Quit Playing Games With My Heart are:
Number 1:
She Quit Me with 0.425 similarity score
--------------------
Number 2:
Love Games with 0.410 similarity score
--------------------
Number 3:
Head Games with 0.390 similarity score
--------------------
Number 4:
After Your Love Is Gone with 0.321 similarity score
--------------------
Number 5:
Baby with 0.316 similarity score
--------------------
Number 6:
Should I, Would I, Could I with 0.290 similarity score
--------------------
Number 7:
Don't Break My Heart with 0.288 similarity score
--------------------
Number 8:
Baby Hold On with 0.280 similarity score
--------------------
Number 9:
That's It I Quit I'm Movin' On with 0.279 similarity score
--------------------
Number 10:
For The Heart with 0.279 similarity score
--------------------
